# Logic Graph Examples

Create interactive graphs of logical formulas with inference edges.

**Smart Sampling:** Use the `max_nodes` parameter to intelligently limit graph size.
The sampler prioritizes: axioms → entailed formulas → tautologies → false statements (simpler formulas first).

For higher depths (3-4), try `max_nodes=200-300` for fast, focused visualizations!

In [3]:
# Example 1: Modus Ponens - x, (x → y) ⊢ y
from boolean_functions import Var, Implies, build_logic_graph, export_to_html

x = Var("x")
y = Var("y")
axioms = [x, Implies(x, y)]

# Build graph with smart sampling (limit to 100 most important nodes)
g = build_logic_graph(["x", "y"], axioms, max_depth=3, max_nodes=300)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check if y is entailed (it should be via modus ponens)
if str(y) in g.nodes:
    print(f"✓ y is entailed: {g.nodes[str(y)]['entailed']}")

# Export to interactive HTML (automatically saved in graphs/ folder)
export_to_html(g, "modus_ponens.html")

Graph has 300 nodes and 480 edges
✓ y is entailed: True
Interactive graph saved to graphs\modus_ponens.html


In [2]:
# Example 2: Multiple inference rules in one graph
from boolean_functions import Var, Implies, build_logic_graph, export_to_html

p = Var("p")
q = Var("q")
r = Var("r")

# Axioms: p, (p → q), (q → r)
# This will show MP, HS, and other inference edges
axioms = [p, Implies(p, q), Implies(q, r)]

# Try higher depth with sampling! max_nodes keeps it manageable
g = build_logic_graph(["p", "q", "r"], axioms, max_depth=2, max_nodes=200)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")

# Check derived formulas
for formula_str in ["q", "r", "(p → r)"]:
    if formula_str in g.nodes:
        entailed = g.nodes[formula_str]['entailed']
        print(f"✓ {formula_str} entailed: {entailed}")

export_to_html(g, "complex.html")

Graph has 200 nodes and 264 edges
✓ q entailed: True
✓ r entailed: True
✓ (p → r) entailed: True
Interactive graph saved to graphs\complex.html


In [3]:
# Example 3: Explore formulas with specific axioms
from boolean_functions import Var, And, build_logic_graph, export_to_html

a = Var("a")
b = Var("b")

# Start with a conjunction axiom
axioms = [And(a, b)]

# Try depth 3 with smart sampling!
g = build_logic_graph(["a", "b"], axioms=axioms, max_depth=3, max_nodes=200)

print(f"Total formulas: {g.number_of_nodes()}")

# Count by category
axiom_count = sum(1 for _, d in g.nodes(data=True) if d.get('is_axiom'))
entailed_count = sum(1 for _, d in g.nodes(data=True) if d.get('entailed'))
tautologies = sum(1 for _, d in g.nodes(data=True) if d.get('tautology'))
print(f"Axioms: {axiom_count}")
print(f"Entailed formulas: {entailed_count}")
print(f"Tautologies: {tautologies}")

print("\n→ Look for conjunction elimination (∧E) edges in green!")

export_to_html(g, "conjunction.html")

Total formulas: 200
Axioms: 1
Entailed formulas: 200
Tautologies: 48

→ Look for conjunction elimination (∧E) edges in green!
Interactive graph saved to graphs\conjunction.html


In [2]:
# Example 4: Minimal Complete Axiom Set for Classical Propositional Logic
# A1: A → (B → A)
# A2: (A → (B → C)) → ((A → B) → (A → C))
# A3: (¬B → ¬A) → (A → B)
# These three axioms plus Modus Ponens are sound and complete!

from boolean_functions import Var, Implies, Not, build_logic_graph, export_to_html

A = Var("A")
B = Var("B")
C = Var("C")

# Define the three axiom schemata
A1 = Implies(A, Implies(B, A))
A2 = Implies(Implies(A, Implies(B, C)), Implies(Implies(A, B), Implies(A, C)))
A3 = Implies(Implies(Not(B), Not(A)), Implies(A, B))

axioms = [A1, A2, A3]

# Build graph with these minimal axioms
g = build_logic_graph(["A", "B", "C"], axioms, max_depth=2)

print(f"Graph has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")
print(f"\nMinimal Complete Axiom Set:")
print(f"A1: A → (B → A)")
print(f"A2: (A → (B → C)) → ((A → B) → (A → C))")
print(f"A3: (¬B → ¬A) → (A → B)")
print(f"\nThese axioms + Modus Ponens are:")
print(f"✓ Sound: only derives tautologies")
print(f"✓ Complete: every tautology can be derived from them")

# Verify they are all tautologies
for i, axiom_str in enumerate([str(A1), str(A2), str(A3)], 1):
    if axiom_str in g.nodes:
        is_taut = g.nodes[axiom_str].get('tautology', False)
        print(f"A{i} is tautology: {is_taut}")

export_to_html(g, "weak_axioms.html")

Graph has 3303 nodes and 1976 edges

Minimal Complete Axiom Set:
A1: A → (B → A)
A2: (A → (B → C)) → ((A → B) → (A → C))
A3: (¬B → ¬A) → (A → B)

These axioms + Modus Ponens are:
✓ Sound: only derives tautologies
✓ Complete: every tautology can be derived from them
A1 is tautology: True
Interactive graph saved to graphs\weak_axioms.html


## Understanding the Graph

**Node colors:**
- 🟠 Orange: Axiom formulas (starting points)
- 🟢 Light Green: Tautologies (always true)
- 🔵 Sky Blue: Entailed from axioms
- 🔴 Red: Not entailed (false under axioms)

**Edge colors (inference rules):**
- Dark Gray-Blue: General entailment
- Hot Pink: Modus Ponens (MP)
- Deep Purple: Modus Tollens (MT)
- Amber: Disjunctive Syllogism (DS)
- Cyan: Hypothetical Syllogism (HS)
- Indigo: Conjunction Elimination Left (∧E-L)
- Brown: Conjunction Elimination Right (∧E-R)

**Interactive features:**
- Drag nodes to rearrange
- Hover over nodes/edges for details
- Scroll to zoom
- Use navigation buttons in bottom-right

**Smart Sampling:**
- Perfect for exploring depth 3-4 without overwhelming the browser!
- Set `max_nodes` to limit graph size
- Prioritizes: axioms → entailed → tautologies → false statements (simpler first)